In [57]:
import pandas as pd
from collections import OrderedDict
import pytz
import helper
import pnf

# Collect ETF data from API and create Pandas Panel
Grab Sector ETF data from Alpha Vantage API and then save each ETF's data into a separate date-indexed CSV file.

In [4]:
etf_dfs = helper.get_sector_etf_dfs()

In [22]:
# Create OrderedDict out of etf_dfs
tickers = etf_dfs.columns
data = OrderedDict()
minor_axis_fields = []
for ticker in tickers:
    data[ticker] = etf_dfs[ticker][0]
    if len(minor_axis_fields) == 0:
        minor_axis_fields = data[ticker].columns
        
# Create panel out of OrderedDict
panel = pd.Panel(data)
panel.minor_axis = minor_axis_fields
panel.major_axis = pd.to_datetime(panel.major_axis)
panel.major_axis = panel.major_axis.tz_localize(pytz.utc)

<class 'pandas.core.panel.Panel'>
Dimensions: 9 (items) x 5072 (major_axis) x 5 (minor_axis)
Items axis: XLK to XLI
Major_axis axis: 1998-12-22 00:00:00+00:00 to 2019-02-20 00:00:00+00:00
Minor_axis axis: Open to Volume


# Generate Trading Strategy
Create the trading strategy for a particular year. To do this, we will determine which sector ETF is the strongest performer on a monthly basis. This analysis will be done at the end of the month to inform trading for the following month. We will invest 100% of our portfolio into the best strongest ETF each month based on relative strength. At the end of the month, if the analysis shows that the strongest ETF is different that the ETF we are currently invested in, we will divest all money in the current ETF and invest 100% of capital into the ETF that has overtaken old ETF as the strongest.

In [69]:
def create_monthly_date_ranges(start_year, end_year):
    '''
        Create two lists representing the start/end dates of each month between start/end year, inclusive.
        
        @param start_year: (int) the first year in range
        @param end_year: (int) the last yaer in range
        
        returns ([str]), ([str]): two lists -- the first is a list of start dates, the second is corresponding end
                                dates. The format is YYYY-MM-DD.
    '''
    monthly_date_ranges = [
        ('01-01', '01-31'),
        ('02-01', '02-29'),
        ('03-01', '03-31'),
        ('04-01', '04-30'),
        ('05-01', '05-31'),
        ('06-01', '06-30'),
        ('07-01', '07-31'),
        ('08-01', '08-31'),
        ('09-01', '09-30'),
        ('10-01', '10-31'),
        ('11-01', '11-30'),
        ('12-01', '12-31')
    ]

    years = [str(x) for x in range(start_year, (end_year + 1))]
    monthly_start_dates = []
    monthly_end_dates = []
    # Create start/end dates for each month
    for year in years:
        for date_range in monthly_date_ranges:
            monthly_start_dates.append(year + '-' + date_range[0])
            monthly_end_dates.append(year + '-' + date_range[1])
                                   
    return monthly_start_dates, monthly_end_dates

In [71]:
# Run through each month in date range and get highest RS ETF for each month
start_year = 1999
end_year = 2000
monthly_start_dates, monthly_end_dates = create_monthly_date_ranges(start_year, end_year)
monthly_best_performers = OrderedDict()

for idx in range(len(monthly_start_dates)):
    start_date = monthly_start_dates[idx]
    end_date = monthly_end_dates[idx]
    
    # Construct DF for this month
    df = etf_dfs.copy()
    for symbol in df.columns:
        df[symbol][0] = df[symbol][0][start_date : end_date]
        
    # Create RS matrix for this month
    rs_matrix = pnf.run_rs_matrix(df.columns, df)
    
    # Select the sector with the highest total and append to best performers dict. The key for
    # this dict is the start date of the next month
    best_sector = rs_matrix['total'].idxmax()
    if idx != (len(monthly_start_dates) - 1):
        monthly_best_performers[monthly_start_dates[idx + 1]] = best_sector
    
    print('Running RS matrix for {0} - {1}'.format(start_date, end_date), end='\r', flush=True)
    
print(monthly_best_performers)

OrderedDict([('1999-02-01', 'XLK'), ('1999-03-01', 'XLF'), ('1999-04-01', 'XLK'), ('1999-05-01', 'XLB'), ('1999-06-01', 'XLE'), ('1999-07-01', 'XLK'), ('1999-08-01', 'XLE'), ('1999-09-01', 'XLP'), ('1999-10-01', 'XLU'), ('1999-11-01', 'XLF'), ('1999-12-01', 'XLE'), ('2000-01-01', 'XLI'), ('2000-02-01', 'XLU'), ('2000-03-01', 'XLE'), ('2000-04-01', 'XLB'), ('2000-05-01', 'XLF'), ('2000-06-01', 'XLF'), ('2000-07-01', 'XLP'), ('2000-08-01', 'XLF'), ('2000-09-01', 'XLK'), ('2000-10-01', 'XLU'), ('2000-11-01', 'XLB'), ('2000-12-01', 'XLF')])
